In [1]:
import os.path

from shared_code.utility.schemas.spark_table_schema import image_table_schema
from shared_code.utility.spark.set_environ import *

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
set_azure_env()

spark = SparkSession \
	.builder \
	.appName('add-missing-captions') \
	.master("local[6]") \
	.config("spark.cores.max", "1") \
	.config("spark.executor.instances", "1") \
	.config("spark.executor.cores", "1") \
	.config("spark.executor.cores", "1") \
	.config("spark.executor.instances", "1") \
	.config("spark.executor.cores", "1")


spark_context = spark.getOrCreate()


In [3]:
data = []
# if os.path.exists("D:\\data\\processed\\reddit_images_vit_captioned.parquet") and os.path.getsize("D:\\data\\processed\\reddit_images_vit_captioned.parquet") > 0:
# 	data = spark_context.read.parquet("D:\\data\\processed\\reddit_images_vit_captioned.parquet")
# else:
spark_df = spark_context.read.parquet("D:\\data\\processed\\reddit_images_processed.parquet", schema=image_table_schema)

print(f"Total Number Of Records From Pre-Processing {spark_df.count()}")

display(spark_df.limit(10).toPandas())

Total Number Of Records From Pre-Processing 6501


,PartitionKey,RowKey,image,text,id,author,url,flair,permalink,hash,subreddit,caption,exists,image_name,updated_caption,small_image,curated
0,training,11090tg,D:\data\images\EarthPorn\vc1wh65itqha1.jpg,"Depoe Bay, OR USA [OC] 1080x1920",11090tg,Bpercell,https://i.redd.it/vc1wh65itqha1.jpg,NaN,/r/EarthPorn/comments/11090tg/depoe_bay_or_usa...,31c6df5a2815d8e5e8c85fe51b08460e,EarthPorn,A group of people are sitting on the shore of ...,True,vc1wh65itqha1.jpg,NaN,D:\data\images\EarthPorn\thumbnail\vc1wh65itqh...,False
1,training,11092s0,D:\data\images\SFWRedheads\8lb4z8n3uqha1.jpg,Morning hair xx,11092s0,demidiamond4,https://i.redd.it/8lb4z8n3uqha1.jpg,NaN,/r/SFWRedheads/comments/11092s0/morning_hair_xx/,afe1ae1ae3e663b0ca62cb3b19a5c3ef,SFWRedheads,A girl in a pink dress is smiling.,True,8lb4z8n3uqha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\8lb4z8n3u...,False
2,training,110961d,D:\data\images\CityPorn\2czgh0e5vqha1.jpg,St Paul’s Cathedral. London.,110961d,OkCardiologist2765,https://i.redd.it/2czgh0e5vqha1.jpg,NaN,/r/CityPorn/comments/110961d/st_pauls_cathedra...,ef3c3f82ec490f8d3ccc06948258a3b4,CityPorn,A man in a red coat is standing in front of a ...,True,2czgh0e5vqha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\2czgh0e5vqha...,False
3,training,1109a10,D:\data\images\SFWRedheads\hy9j0og8wqha1.jpg,Do you like my natural red curls?!,1109a10,ketunicorn,https://i.redd.it/hy9j0og8wqha1.jpg,NaN,/r/SFWRedheads/comments/1109a10/do_you_like_my...,9008fe70d17b904fcfc7c6a2232ad569,SFWRedheads,A girl in a pink dress is standing in front of...,True,hy9j0og8wqha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\hy9j0og8w...,False
4,training,1109ycm,D:\data\images\CityPorn\7beed5n54rha1.jpg,"Taipei, the Seoul of Taiwan (which makes Kaohs...",1109ycm,Ducky118,https://i.redd.it/7beed5n54rha1.jpg,NaN,/r/CityPorn/comments/1109ycm/taipei_the_seoul_...,848b795097dca410c409797c679ab4fa,CityPorn,A bird flies over the water.,True,7beed5n54rha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\7beed5n54rha...,False
5,training,110b0wt,D:\data\images\SFWNextDoorGirls\6piZs7j.jpg,"A real no makeup moment, cause this is how Sun...",110b0wt,princessxo699,https://i.imgur.com/6piZs7j.jpg,Feelin Happy 😃,/r/SFWNextDoorGirls/comments/110b0wt/a_real_no...,5ccd774afb17fb0c44460121ed9ce61d,SFWNextDoorGirls,A girl in a pink hat and sunglasses is talking...,True,6piZs7j.jpg,NaN,D:\data\images\SFWNextDoorGirls\thumbnail\6piZ...,False
6,training,110b1an,D:\data\images\CityPorn\rgotpnatzpha1.jpg,"Duisburg, Detroit of Germany",110b1an,Expensive-Team7416,https://i.redd.it/rgotpnatzpha1.jpg,NaN,/r/CityPorn/comments/110b1an/duisburg_detroit_...,3eb930bdf3fac4e5b16617edad9df65a,CityPorn,A bird flies over the water.,True,rgotpnatzpha1.jpg,NaN,D:\data\images\CityPorn\thumbnail\rgotpnatzpha...,False
7,training,110bipg,D:\data\images\SFWNextDoorGirls\xywgncrbnrha1.jpg,enjoy your Sunday!,110bipg,trickyalice,https://i.redd.it/xywgncrbnrha1.jpg,Outfit of the Day,/r/SFWNextDoorGirls/comments/110bipg/enjoy_you...,9475adbe05427c47772346f8ab672952,SFWNextDoorGirls,A woman in a pink shirt and jeans is talking t...,True,xywgncrbnrha1.jpg,NaN,D:\data\images\SFWNextDoorGirls\thumbnail\xywg...,False
8,training,110c2gi,D:\data\images\SFWRedheads\99hm504mtrha1.jpg,Happy sunday! Sending greetings! ❤️,110c2gi,GingeMariia,https://i.redd.it/99hm504mtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2gi/happy_sunday_s...,20ced2827eb9e9cd3464835306cfd075,SFWRedheads,A girl in a pink shirt and blue jeans is stand...,True,99hm504mtrha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\99hm504mt...,False
9,training,110c2vo,D:\data\images\SFWRedheads\kkerxboqtrha1.jpg,The kind of girl you can have a beer with.,110c2vo,MissKatiesss,https://i.redd.it/kkerxboqtrha1.jpg,NaN,/r/SFWRedheads/comments/110c2vo/the_kind_of_gi...,f53572296a0b8fb6654808866440316a,SFWRedheads,A woman in a red dress is talking to a man in ...,True,kkerxboqtrha1.jpg,NaN,D:\data\images\SFWRedheads\thumbnail\kkerxboqt...,False


In [4]:
from shared_code.utility.scripts.image_analysis import ImageCaption
import random
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
from shared_code.utility.storage.table import TableAdapter

table_adapter: TableAdapter = TableAdapter()
caption_instance = ImageCaption()


def get_caption_image_name(image_path: str, caption: str, row_key: str, partition_key: str):
	if not os.path.exists(image_path):
		return ""
	try:
		if caption is None or caption == "" or caption == "NaN":
			print(f"Caption is empty for image {image_path} - {caption}")
			caption = caption_instance.caption_image_vit(image_path, random.choice([1, 2]))
			print(f"Caption is completed for image {image_path} - {caption}")
			entity = table_adapter.get_entity("training", partition_key, row_key)
			entity["updated_caption"] = caption
			table_adapter.upsert_entity_to_table("training", entity)
			return caption
		else:
			return caption
	except:
		return ""

In [ ]:
get_caption_image_name_udf = udf(get_caption_image_name, StringType())

print(spark_df.where("updated_caption == 'NaN'").count())

spark_df.withColumn("updated_caption", get_caption_image_name_udf("small_image", "updated_caption", "RowKey", "PartitionKey")).write.parquet("D:\\data\\processed\\reddit_images_vit_captioned.parquet", mode="append")

4700


In [ ]:
new_frame = spark_context.createDataFrame(spark.read.parquet("D:\\data\\processed\\reddit_images_vit_captioned.parquet").collect(), schema=image_table_schema)

display(new_frame.limit(10).toPandas())

In [ ]:
from shared_code.utility.storage.table import TableAdapter

table_adapter: TableAdapter = TableAdapter()

records = new_frame.toPandas().to_dict('records')

for record in records:
	table_adapter.upsert_entity_to_table("training", record)